In [1]:
# Imports
import pandas as pd
import numpy as np

# `numexpr` package
1. Efficient computational time: does not create temporary intermediate memory, unlike normal df calculations
2. Efficient computational memory: uses less memory than typical df calculations. With less L1, L2 data or physical memory needed, it can handle larger df

**pitfall: less efficient if df is small**
  
## `numexpr.evaluate()`
1. numpy-style expression

In [6]:
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)

In [8]:
mask = (x > 0.5) & (y < 0.5) # BTS: creates tmp1 for (x > 0.5), tmp2 for (y < 0.5) before creating mask

import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

## `eval()`
1. Arithmetic
2. Comparison
3. Bitwise when querying or arithmetic

In [12]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                        for i in range(5))

In [13]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

In [14]:
# 1. Arithmetic
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

In [15]:
# 2. Comparison
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

In [18]:
# 3. Bitwise
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [17]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

In [19]:
# 4. Indexing and .attr
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

In [20]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [21]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [22]:
# More succinct representation
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

In [23]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [24]:
# Local variables with `@`: pandas.eval() cannot use
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

In [25]:
# pd.query()
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

## `query()`
1. **Filtering**: query values based on many conditions.

In [26]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True